In [3]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 7.2 MB/s eta 0:00:00


In [100]:
import zipfile
import os

zip_dir = "/content/sim 2"
for filename in os.listdir(zip_dir):
    if filename.endswith('.zip'):
        zip_path = os.path.join(zip_dir, filename)

        # Folder name = zip file name (without .zip)
        extract_dir = os.path.join(zip_dir, filename[:-4])
        os.makedirs(extract_dir, exist_ok=True)

        # Extract ZIP file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
            print(f'Extracted {filename} to {extract_dir}')

Extracted ro-crate-metadata-012094b0895944ae.zip to /content/sim 2/ro-crate-metadata-012094b0895944ae
Extracted ro-crate-metadata-8d51b6911e5c3560.zip to /content/sim 2/ro-crate-metadata-8d51b6911e5c3560


In [101]:
from rdflib import Graph

base_dir = "/content/sim 2"

graph_list = []

for root, dirs, files in os.walk(base_dir):
    if 'provenance.ttl' in files:
        file_path = os.path.join(root, 'provenance.ttl')
        try:
            g = Graph()
            g.parse(file_path, format='ttl')
            graph_list.append(g)
            print(f"✅ Parsed: {file_path}")
        except Exception as e:
            print(f"❌ Failed to parse {file_path}: {e}")

print(f"\nTotal graphs loaded: {len(graph_list)}")


✅ Parsed: /content/sim 2/ro-crate-metadata-8d51b6911e5c3560/provenance.ttl
✅ Parsed: /content/sim 2/ro-crate-metadata-012094b0895944ae/provenance.ttl

Total graphs loaded: 2


In [103]:
from rdflib import Graph, Namespace, Literal
from tabulate import tabulate

query = """
PREFIX cr: <http://mlcommons.org/croissant/>
PREFIX sio: <http://semanticscience.org/resource/>

SELECT ?value_element_size ?value_young_modulus ?value_poisson_ratio ?value_max_mises_stress
WHERE {
  ?max_mises_stress cr:source/cr:extract [ cr:jsonPath "/max_mises_stress" ] ;
             sio:SIO_000210 [ schema:value ?value_max_mises_stress ] .

  ?poisson_ratio cr:source/cr:extract [ cr:jsonPath "/poisson-ratio/value" ] ;
             sio:SIO_000210 [ schema:value ?value_poisson_ratio ] .

  ?young_modulus cr:source/cr:extract [ cr:jsonPath "/young-modulus/value" ] ;
             sio:SIO_000210 [ schema:value ?value_young_modulus ] .

  ?field_size cr:source/cr:extract [ cr:jsonPath "/element-size/value" ] ;
             sio:SIO_000210 [ schema:value ?value_element_size ] .

  FILTER EXISTS {
    ?field_degree cr:source/cr:extract [ cr:jsonPath "/element-degree" ] ;
                  sio:SIO_000210 [ schema:value 1 ] .
  }

  FILTER EXISTS {
    ?field_order cr:source/cr:extract [ cr:jsonPath "/element-order" ] ;
                 sio:SIO_000210 [ schema:value 1 ] .
  }
}
"""

table_data = []

for g in graph_list:
    local_ns = dict(g.namespace_manager.namespaces()).get("local")
    hash_id = local_ns.strip("/").split("/")[-1] if local_ns else "UNKNOWN"

    results = g.query(query)

    for row in results:
        value_element_size = row.value_element_size
        value_young_modulus = row.value_young_modulus
        value_poisson_ratio = row.value_poisson_ratio
        value_max_mises_stress = row.value_max_mises_stress
        table_data.append(
            [
                hash_id,
                value_element_size,
                value_young_modulus,
                value_poisson_ratio,
                value_max_mises_stress
            ]
        )

print(
    tabulate(
        table_data,
        headers=[
            "Simulation Hash",
            "element-size",
            "young-modulus",
            "poisson-ratio",
            "max-mises-stress"
        ],
    )
)


Simulation Hash      element-size    young-modulus    poisson-ratio    max-mises-stress
-----------------  --------------  ---------------  ---------------  ------------------
8d51b6911e5c3560         0.003125          2.1e+11              0.3         2.99783e+08
012094b0895944ae         0.0125            2.1e+11              0.3         3.00129e+08
